## Deliverable 2. Create a Customer Travel Destinations Map.

In [56]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Amahai,ID,-3.3333,128.9167,83.07,73,41,5.30,scattered clouds
1,1,Gubkinskiy,RU,64.4340,76.5026,58.37,60,83,12.66,broken clouds
2,2,Salisbury,GB,51.0693,-1.7957,60.10,93,77,6.49,broken clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,37.51,75,0,23.02,clear sky
4,4,Fortuna,US,40.5982,-124.1573,55.92,91,90,6.91,overcast clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Amahai,ID,-3.3333,128.9167,83.07,73,41,5.30,scattered clouds
5,5,Rikitea,PF,-23.1203,-134.9692,72.61,79,73,8.46,broken clouds
10,10,La Ronge,CA,55.1001,-105.2842,70.05,49,40,3.44,smoke
17,17,Hilo,US,19.7297,-155.0900,74.93,88,90,5.75,haze
22,22,Vaini,TO,-21.2000,-175.2000,77.16,88,75,20.71,moderate rain
...,...,...,...,...,...,...,...,...,...,...
705,705,Praia Da Vitoria,PT,38.7333,-27.0667,71.76,90,66,6.98,broken clouds
706,706,Pyay,MM,18.8167,95.2167,87.42,67,84,6.85,broken clouds
707,707,Artyom,AZ,40.4723,50.3330,78.94,68,96,8.77,overcast clouds
710,710,Maine-Soroa,NE,13.2114,12.0241,77.32,75,69,9.17,broken clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                363
City                   363
Country                362
Lat                    363
Lng                    363
Max Temp               363
Humidity               363
Cloudiness             363
Wind Speed             363
Current Description    363
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna() # One Nan Value dropped
clean_df.count()

City_ID                362
City                   362
Country                362
Lat                    362
Lng                    362
Max Temp               362
Humidity               362
Cloudiness             362
Wind Speed             362
Current Description    362
dtype: int64

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Amahai,ID,83.07,scattered clouds,-3.3333,128.9167,
5,Rikitea,PF,72.61,broken clouds,-23.1203,-134.9692,
10,La Ronge,CA,70.05,smoke,55.1001,-105.2842,
17,Hilo,US,74.93,haze,19.7297,-155.0900,
22,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,
23,Mahibadhoo,MV,82.53,overcast clouds,3.7833,72.9667,
25,Addanki,IN,86.05,overcast clouds,15.8167,79.9833,
27,College,US,73.38,clear sky,64.8569,-147.8028,
29,Hubli,IN,73.44,light intensity drizzle,15.3500,75.1667,
31,Freeport,US,71.56,clear sky,40.6576,-73.5832,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [60]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True) # Replacing blank spaces with NAN values to delete them
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Amahai,ID,83.07,scattered clouds,-3.3333,128.9167,HOTEL ISABELA
5,Rikitea,PF,72.61,broken clouds,-23.1203,-134.9692,People ThankYou
10,La Ronge,CA,70.05,smoke,55.1001,-105.2842,Northland Motor Hotel
17,Hilo,US,74.93,haze,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
705,Praia Da Vitoria,PT,71.76,broken clouds,38.7333,-27.0667,Hotel Branco I
706,Pyay,MM,87.42,broken clouds,18.8167,95.2167,Shwe Pyi Thar
707,Artyom,AZ,78.94,overcast clouds,40.4723,50.3330,Merdan Respect
710,Maine-Soroa,NE,77.32,broken clouds,13.2114,12.0241,Complexe


In [62]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [63]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Lat/dt><dd>{Lat} °</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [67]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))